In [38]:
from random import random, sample
import numpy as np

from astropy import coordinates
from astroquery.esa.hubble import ESAHubbleClass
from astropy.io import fits
from astroquery.mast import Observations
from astropy.table import vstack, Table

import matplotlib.pyplot as plt
from PIL import Image

import sys
import os
import time
import re
import json
 
import requests
from urllib.parse import quote as urlencode

In [37]:
#astroquery.mast.Observations sınıfını kullanarak istenilen özelliklere sahip Hubble verisi sorgulayan
#ve daha sonra indirilmek üzere .txt dosyası olarak kaydeden fonksiyon
def query_hubble(coords: list):

    prod_table = None

    flag = True
    for i in range(len(coords)):
        
        #Gökte 2 derecelik yarıçapta istenilen kriterlere uygun sorgu yapılır ve sonuçlar alınır.
        table = Observations.query_criteria(
                                          **{"coordinates": str(coords[i].ra) + " " + str(coords[i].dec),
                                          "radius": "2.0 deg",
                                          "pagesize": 10000, 
                                          "page": 1,
                                          "obs_collection": ["HST"],
                                          "dataproduct_type": ["IMAGE"],
                                          "wavelength_region": ["OPTICAL"],
                                          "intentType": ["science"],
                                          "dataRights": ["PUBLIC"]}
                                         )

        #Ekstra filtreleme
        if(len(table) != 0):
            temp_products = Observations.get_product_list(table)
            temp_products = temp_products[temp_products["productType"] == "SCIENCE"]
            temp_products = temp_products[temp_products["calib_level"] == 3]
            temp_products = temp_products[temp_products["productSubGroupDescription"] == "DRZ"]
            temp_products = temp_products[temp_products["obs_collection"] != "HLA"]
            temp_products = temp_products[temp_products["filters"] != "detection"]
            temp_products = temp_products[temp_products["filters"] != "DETECTION"]

            
            if flag:
                flag = False
                prod_table = temp_products
        
            else:
                prod_table = vstack([prod_table, temp_products])

            print(len(prod_table))
        
        else:
            print("Gözlem verisi yok!")

        print(i)

    #Tablo daha sonra download_products() fonksiyonu ile indirilmek üzere belleğe kaydedilir.    
    prod_table.write("download_table.txt", format='ascii')
    return prod_table